In [1]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [6]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
headers = {"X-Naver-Client-Id": "376ZADkaoIoxnJh_ixoG",
           "X-Naver-Client-Secret":"FEja8f5GwD"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [3]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"",
    "ages":["40"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='40'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='40'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='40'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["디지털/가전"], axis=1, inplace=True)
df_all_a

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,4.94294,0.30865,2.49175,2.67368,10.50940,5.75464,0.97829,0.40126,1.17389,0.38310,...,3.73455,0.97829,0.61235,1.14106,1.88331,0.06653,0.08731,PC+MO,전체,40
2021-05-11,4.75047,0.28774,2.16298,2.55683,9.25898,5.22802,0.83273,0.38785,1.15693,0.38185,...,3.48526,0.83273,0.59002,1.10281,1.56995,0.07069,0.08827,PC+MO,전체,40
2021-05-12,4.94815,0.29599,1.96605,2.51554,8.84318,5.65003,0.79891,0.36677,1.18109,0.38943,...,3.42952,0.79891,0.55661,1.03278,1.43751,0.06594,0.08365,PC+MO,전체,40
2021-05-13,4.58758,0.26745,1.83886,2.55729,8.51448,7.83034,0.72638,0.32927,1.07586,0.34735,...,3.21330,0.72638,0.51199,0.98492,1.32778,0.06461,0.07432,PC+MO,전체,40
2021-05-14,4.24559,0.24475,1.67210,2.36336,7.73115,9.57868,0.67309,0.30890,0.95013,0.31474,...,3.05633,0.67309,0.46184,0.91289,1.28966,0.06532,0.06690,PC+MO,전체,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,5.17514,0.24432,1.67689,1.45790,6.53835,9.84652,0.53623,0.19027,0.77981,0.27229,...,2.41165,0.53623,0.49954,0.82293,1.28625,0.06878,0.10323,MO,전체,40
2023-06-30,4.57650,0.21739,1.54057,1.33280,5.80260,8.26709,0.53318,0.16976,0.69676,0.24756,...,2.22992,0.53318,0.45923,0.79445,1.22534,0.05931,0.09408,MO,전체,40
2023-07-01,5.75478,0.23405,1.91694,1.59820,7.03231,9.95388,0.67207,0.18464,0.68418,0.25380,...,2.66842,0.67207,0.56509,0.82504,1.62199,0.07395,0.09309,MO,전체,40


In [4]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"f",
    "ages":["40"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='40'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='40'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='40'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["디지털/가전"], axis=1, inplace=True)
df_all_b

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,3.14684,0.15883,1.05760,1.47824,9.72007,4.07286,0.44839,0.12410,0.14499,0.16131,...,1.84486,0.44839,0.38281,0.21637,0.79301,0.02892,0.06520,PC+MO,여성,40
2021-05-11,3.01167,0.15105,0.87676,1.33052,8.69034,3.75098,0.39938,0.12669,0.13238,0.14425,...,1.70401,0.39938,0.35753,0.19363,0.61489,0.02762,0.06619,PC+MO,여성,40
2021-05-12,3.19412,0.17076,0.82033,1.33090,8.20938,3.96470,0.36495,0.10834,0.14746,0.16792,...,1.67447,0.36495,0.34666,0.19184,0.56903,0.02632,0.06872,PC+MO,여성,40
2021-05-13,3.02082,0.14394,0.74950,1.39863,7.91395,5.83800,0.36625,0.11569,0.14035,0.15543,...,1.56989,0.36625,0.32020,0.19907,0.49357,0.02843,0.05797,PC+MO,여성,40
2021-05-14,2.73454,0.12688,0.67163,1.21643,7.12594,7.42582,0.31940,0.10123,0.12336,0.12861,...,1.48108,0.31940,0.28287,0.16971,0.49109,0.02960,0.04573,PC+MO,여성,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,3.24061,0.12330,0.69164,1.38542,5.62877,7.47859,0.23138,0.05209,0.09350,0.11034,...,1.11094,0.23138,0.30493,0.14955,0.33325,0.02876,0.07419,MO,여성,40
2023-06-30,2.75367,0.12071,0.62788,1.25258,4.99130,6.09932,0.24039,0.04432,0.07808,0.10898,...,0.97428,0.24039,0.27486,0.14274,0.31659,0.02967,0.06861,MO,여성,40
2023-07-01,3.69536,0.11488,0.78812,1.47095,5.87208,7.10108,0.29456,0.04989,0.07289,0.10224,...,1.17632,0.29456,0.33428,0.15739,0.47223,0.03311,0.06492,MO,여성,40


In [7]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"m",
    "ages":["40"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='40'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='40'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='40'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["디지털/가전"], axis=1, inplace=True)
df_all_c

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,7.44108,0.53159,4.69091,4.42296,10.43601,7.93554,1.78276,0.83755,2.83783,0.72352,...,6.56881,1.78276,0.93433,2.62438,3.55674,0.12402,0.11436,PC+MO,남성,40
2021-05-11,7.17379,0.49029,4.14535,4.37737,8.96897,7.11897,1.48604,0.79756,2.81553,0.75054,...,6.16179,1.48604,0.92038,2.56396,3.04700,0.13732,0.11513,PC+MO,남성,40
2021-05-12,7.37077,0.47700,3.72636,4.26784,8.72740,7.85194,1.45802,0.77460,2.85244,0.72846,...,6.06731,1.45802,0.85161,2.38247,2.77928,0.12710,0.09843,PC+MO,남성,40
2021-05-13,6.72799,0.44942,3.51686,4.25752,8.38574,10.27166,1.26445,0.66265,2.58758,0.63969,...,5.68303,1.26445,0.78097,2.24339,2.62405,0.11985,0.09293,PC+MO,남성,40
2021-05-14,6.33503,0.41987,3.21552,4.06999,7.72077,12.05970,1.20722,0.63464,2.28624,0.60135,...,5.42695,1.20722,0.71506,2.10563,2.52792,0.11963,0.09513,PC+MO,남성,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,7.32141,0.40133,3.04442,1.17680,6.37333,11.46842,0.95439,0.39490,1.84344,0.49790,...,4.16025,0.95439,0.72153,1.84915,2.70491,0.12414,0.12815,MO,남성,40
2023-06-30,6.68381,0.33647,2.81083,1.10186,5.66357,9.96631,0.92949,0.35639,1.65734,0.43656,...,3.94585,0.92949,0.67355,1.78902,2.57797,0.09790,0.11491,MO,남성,40
2023-07-01,7.96957,0.39052,3.48499,1.37978,7.19483,12.45513,1.18749,0.38409,1.63451,0.46535,...,4.70116,1.18749,0.83596,1.84076,3.31360,0.12936,0.11928,MO,남성,40


In [8]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7065 entries, 2021-05-10 to 2023-07-03
Data columns (total 28 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   휴대폰액세서리    7065 non-null   float64
 1   생활가전       7065 non-null   float64
 2   영상가전       7065 non-null   float64
 3   이미용가전      7065 non-null   float64
 4   주방가전       7065 non-null   float64
 5   계절가전       7065 non-null   float64
 6   카메라/캠코더용품  7065 non-null   float64
 7   PC         7065 non-null   float64
 8   PC부품       7065 non-null   float64
 9   PC액세서리     7065 non-null   float64
 10  저장장치       7065 non-null   float64
 11  멀티미디어장비    7065 non-null   float64
 12  모니터        7065 non-null   float64
 13  모니터주변기기    7065 non-null   float64
 14  노트북        7065 non-null   float64
 15  노트북액세서리    7065 non-null   float64
 16  소프트웨어      7065 non-null   float64
 17  주변기기       7065 non-null   float64
 18  음향가전       7065 non-null   float64
 19  태블릿PC      7065 non-null   float64
 20

In [9]:
## 엑셀에 저장
df_all.to_excel("MCLS_04_all_fm_40.xlsx", sheet_name="click ratio")
df_all.head()

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,4.94294,0.30865,2.49175,2.67368,10.50940,5.75464,0.97829,0.40126,1.17389,0.38310,...,3.73455,0.97829,0.61235,1.14106,1.88331,0.06653,0.08731,PC+MO,전체,40
2021-05-11,4.75047,0.28774,2.16298,2.55683,9.25898,5.22802,0.83273,0.38785,1.15693,0.38185,...,3.48526,0.83273,0.59002,1.10281,1.56995,0.07069,0.08827,PC+MO,전체,40
2021-05-12,4.94815,0.29599,1.96605,2.51554,8.84318,5.65003,0.79891,0.36677,1.18109,0.38943,...,3.42952,0.79891,0.55661,1.03278,1.43751,0.06594,0.08365,PC+MO,전체,40
2021-05-13,4.58758,0.26745,1.83886,2.55729,8.51448,7.83034,0.72638,0.32927,1.07586,0.34735,...,3.21330,0.72638,0.51199,0.98492,1.32778,0.06461,0.07432,PC+MO,전체,40
2021-05-14,4.24559,0.24475,1.67210,2.36336,7.73115,9.57868,0.67309,0.30890,0.95013,0.31474,...,3.05633,0.67309,0.46184,0.91289,1.28966,0.06532,0.06690,PC+MO,전체,40
